In [4]:

from read_avi import *
os.chdir('/Users/davy/PycharmProjects/utd-mhad')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [10]:

# Function to split the data into training and testing sets
def split_data(video_data, labels):
    X_train, X_test, y_train, y_test = train_test_split(video_data, labels, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test


# Function to prepare data for machine learning training
def prepare_data_for_ml(directory, filesuffix):
    video_data, labels = process_avi_files(directory, filesuffix, None)
    # for video_num in range(0, len(video_data)):
    #     video_processor.save_video(video_data[video_num], output_file="out/vid/"+labels[video_num], frame_rate=video_processor.min_frame_count)
    return video_data, labels

# Prepare data for ML training
video_data, labels = prepare_data_for_ml(avi_files_directory, avi_files_suffix)
X_train, X_test, y_train, y_test = split_data(video_data, labels)


Total frames/RGBMPG:  33


In [11]:
# Define the 3D CNN model
def create_model(num_classes):
    # The CNN model
    model = keras.Sequential([
        # First (Conv3D + MaxPool3D) layer
        layers.Conv3D(64, kernel_size=(3, 3, 3), activation="relu", padding="same", input_shape=(None, 224, 224, 3)),
        layers.MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), padding="valid"),

        # Second (Conv3D + MaxPool3D) layer
        # Repeat this block with more filters and/or more layers

        # Flatten and Dense layers
        layers.Flatten(),
        layers.Dense(512, activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation="softmax")  # num_classes is the number of actions
    ])

    return model

# Assuming num_classes is the number of actions (27 in your case)
num_classes = 27
model = create_model(num_classes)

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Train the model
# X_train, y_train, X_val, y_val are your training and validation datasets
history = model.fit(X_train, y_train, validation_split=0.2, epochs=20, batch_size=32)

# Evaluate the model
# X_test, y_test are your test datasets
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}, Test loss: {test_loss}")

# Visualization of training
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()

# Confusion matrix
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
confusion_mtx = tf.math.confusion_matrix(y_true, y_pred_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx, annot=True, fmt='g')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

ValueError: The last dimension of the inputs to a Dense layer should be defined. Found None. Full input shape received: (None, None)